In [5]:
import time
import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score, log_loss
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [2]:
# actually training the LightBGM model
xtrain_path = 'models/X_train.npy'
xtest_path = 'models/X_test.npy'
ytrain_path = 'models/y_train.npy'
ytest_path = 'models/y_test.npy'
cat_ind_path = 'models/cat_ind.npy'

light_model_path = 'models/light_trained.pkl'

X_train = np.load(xtrain_path)
X_test = np.load(xtest_path)
y_train = np.load(ytrain_path)
y_test = np.load(ytest_path)
categorical_features_indices = list(np.load(cat_ind_path))
categorical_features_indices = [int(i) for i in categorical_features_indices]

print(f'X_train shape : {X_train.shape}')
print(f'X_test shape : {X_test.shape}')
print(f'y_train shape : {y_train.shape}')
print(f'y_test shape : {y_test.shape}')

verbose = True

X_train shape : (503656, 346)
X_test shape : (125914, 346)
y_train shape : (503656,)
y_test shape : (125914,)


In [3]:
early_stopping_callback = lgb.early_stopping(stopping_rounds = 50, verbose = True)
lgbm = lgb.LGBMClassifier(objective = 'binary',
                          metric = 'binary_logloss',
                          n_estimators = 1000, # tuneable
                          learning_rate = 0.05, # tuneable
                          num_leaves = 31, # tuneable
                          max_depth = -1, # no limit on tree depth (hmmmm....)
                          random_state = 42,
                          n_jobs = -1) # use all cores

param_dist = {
    'n_estimators': sp_randint(100, 1500), # Sample number of estimators
    'learning_rate': sp_uniform(0.01, 0.1), # Sample learning rate
    'num_leaves': sp_randint(20, 60), # Sample number of leaves
    'max_depth': [-1, 10, 15, 20], # List of specific values to try
    'feature_fraction': sp_uniform(0.6, 0.4), # Sample between 0.6 and 1.0 (0.6 + 0.4)
    'bagging_fraction': sp_uniform(0.6, 0.4), # Sample between 0.6 and 1.0
    'bagging_freq': sp_randint(1, 10),
    'min_child_samples': sp_randint(10, 50), # Replaces min_data_in_leaf for sklearn API
    'lambda_l1': sp_uniform(0, 1.0), # L1 regularization
    'lambda_l2': sp_uniform(0, 1.0)  # L2 regularization
}

n_iter_search = 5 # Number of parameter settings that are sampled. 
random_search = RandomizedSearchCV(
    estimator = lgbm, # the model we training
    param_distributions = param_dist,
    n_iter = n_iter_search,
    scoring = 'neg_log_loss', # Use neg_log_loss since lower logloss is better
    cv = 2, # Number of cross-validation folds 
    refit = True, # Refit the best model on the whole training data
    random_state=42,
    verbose=1,
    n_jobs=1
)

fit_params = {
    'categorical_feature' : categorical_features_indices,
}

print('Starting training')
t0 = time.time()

random_search.fit (X_train, y_train, **fit_params)


Starting training
Fitting 2 folds for each of 5 candidates, totalling 10 fits
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.8394633936788146, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8394633936788146
[LightGBM] [Warning] lambda_l2 is set=0.15599452033620265, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.15599452033620265
[LightGBM] [Warning] lambda_l1 is set=0.15601864044243652, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.15601864044243652
[LightGBM] [Warning] bagging_fraction is set=0.749816047538945, subsample=1.0 will be ignored. Current value: bagging_fraction=0.749816047538945
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.8394633936788146, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0

/home/thomas/miniconda3/envs/data/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.8394633936788146, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8394633936788146
[LightGBM] [Warning] lambda_l2 is set=0.15599452033620265, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.15599452033620265
[LightGBM] [Warning] lambda_l1 is set=0.15601864044243652, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.15601864044243652
[LightGBM] [Warning] bagging_fraction is set=0.749816047538945, subsample=1.0 will be ignored. Current value: bagging_fraction=0.749816047538945
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.8394633936788146, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8394633936788146
[LightGBM] [Warning] lambda_l2 is set=0.15599452033620265, r

/home/thomas/miniconda3/envs/data/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.8394633936788146, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8394633936788146
[LightGBM] [Warning] lambda_l2 is set=0.15599452033620265, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.15599452033620265
[LightGBM] [Warning] lambda_l1 is set=0.15601864044243652, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.15601864044243652
[LightGBM] [Warning] bagging_fraction is set=0.749816047538945, subsample=1.0 will be ignored. Current value: bagging_fraction=0.749816047538945
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9879639408647978, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9879639408647978
[LightGBM] [Warning] lambda_l2 is set=0.21233911067827616, r

...

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.6682096494749166, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6682096494749166
[LightGBM] [Warning] lambda_l2 is set=0.9488855372533332, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9488855372533332
[LightGBM] [Warning] lambda_l1 is set=0.06505159298527952, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06505159298527952
[LightGBM] [Warning] bagging_fraction is set=0.9439761626945282, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9439761626945282
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8439986631130484, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8439986631130484
[LightGBM] [Warning] lambda_l2 is set=0.17336465350777208, r

/home/thomas/miniconda3/envs/data/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8439986631130484, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8439986631130484
[LightGBM] [Warning] lambda_l2 is set=0.17336465350777208, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.17336465350777208
[LightGBM] [Warning] lambda_l1 is set=0.8331949117361643, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8331949117361643
[LightGBM] [Warning] bagging_fraction is set=0.8736932106048627, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8736932106048627
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8439986631130484, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8439986631130484
[LightGBM] [Warning] lambda_l2 is set=0.17336465350777208, r

/home/thomas/miniconda3/envs/data/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8439986631130484, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8439986631130484
[LightGBM] [Warning] lambda_l2 is set=0.17336465350777208, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.17336465350777208
[LightGBM] [Warning] lambda_l1 is set=0.8331949117361643, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8331949117361643
[LightGBM] [Warning] bagging_fraction is set=0.8736932106048627, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8736932106048627
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.6682096494749166, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6682096494749166
[LightGBM] [Warning] lambda_l2 is set=0.9488855372533332, re

In [8]:
if verbose:
    print(f'Finished training after {time.time() - t0} seconds.\nBest parameters found are:\n')
    print(random_search.best_params_)

best_lgbm = random_search.best_estimator_
joblib.dump(best_lgbm, light_model_path)
print(f'Best LightGBM model saved at {light_model_path}.')

y_pred = best_lgbm.predict(X_test)

print(f'Classification report\n{classification_report(y_test, y_pred)}')
print(f'AUC SCORES\n{roc_auc_score(y_test, y_pred)}')
print(f'LogLoss score\n{log_loss(y_test, y_pred)}')

Finished training after 992.949738740921 seconds.
Best parameters found are:

{'bagging_fraction': np.float64(0.9439761626945282), 'bagging_freq': 7, 'feature_fraction': np.float64(0.6682096494749166), 'lambda_l1': np.float64(0.06505159298527952), 'lambda_l2': np.float64(0.9488855372533332), 'learning_rate': np.float64(0.10656320330745593), 'max_depth': 10, 'min_child_samples': 18, 'n_estimators': 1469, 'num_leaves': 21}
Best LightGBM model saved at models/light_trained.pkl.
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.6682096494749166, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6682096494749166
[LightGBM] [Warning] lambda_l2 is set=0.9488855372533332, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9488855372533332
[LightGBM] [Warning] lambda_l1 is set=0.06505159298527952, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0650515

/home/thomas/miniconda3/envs/data/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     73937
           1       1.00      1.00      1.00     51977

    accuracy                           1.00    125914
   macro avg       1.00      1.00      1.00    125914
weighted avg       1.00      1.00      1.00    125914

AUC SCORES
0.9998603713958238
LogLoss score
0.0045800979575416735
